This is just a simple ipynb File to only check the working of model on the dataset.
 Developer details: 
        Name: Harshita and Prachi
    Version:
        Version: V 1.0 (20 September 2024)

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from keras.regularizers import l2
import spacy
import re

In [2]:
df_train = pd.read_csv("C:/Users/HP/Desktop/DIY_TransferLearning/Data/TwitterData/twitter_training.csv")
df_test = pd.read_csv("C:/Users/HP/Desktop/DIY_TransferLearning/Data/TwitterData/twitter_validation.csv")

In [3]:
df_train.head()

,tweetID,entity,sentiment,tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tweetID        74682 non-null  int64 
 1   entity         74682 non-null  object
 2   sentiment      74682 non-null  object
 3   tweet_content  73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [5]:
df_train.drop_duplicates(inplace=True)

In [6]:
df_train.dropna(inplace=True)

In [9]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm") 

def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        # Skip stopwords and punctuation
        if not token.is_stop and not token.is_punct:
            filtered_tokens.append(token.lemma_)  # Lemmatize the token
            
    return " ".join(filtered_tokens)
 

In [10]:
df_train['Preprocessed Text'] = df_train['tweet_content'].apply(preprocess) 

In [11]:
df_train

,tweetID,entity,sentiment,tweet_content,Preprocessed Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,m get borderland murder
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,come border kill
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,m get borderland kill
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,m come borderland murder
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,m get borderland 2 murder
...,...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...,realize Windows partition Mac like 6 year Nvid...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...,realize Mac window partition 6 year Nvidia dri...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...,realize windows partition Mac 6 year Nvidia dr...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...,realize windows partition Mac like 6 year Nvid...


In [13]:
# Encoding target column
le = LabelEncoder()
df_train['sentiment'] = le.fit_transform(df_train['sentiment'])

In [14]:
# Split data into train and test

x_train, x_test, y_train, y_test = train_test_split(df_train['Preprocessed Text'], df_train['sentiment'], 
                                                    test_size=0.2, random_state=42, stratify=df_train['sentiment'])

In [15]:
x_test.shape

(14331,)

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# classifier
clf = Pipeline([
    ('vectorizer_tri_grams', TfidfVectorizer()),
    ('naive_bayes', (MultinomialNB()))         
])

In [17]:
clf.fit(x_train, y_train)

Pipeline(steps=[('vectorizer_tri_grams', TfidfVectorizer()),
                ('naive_bayes', MultinomialNB())])

In [18]:
y_pred = clf.predict(x_test)

In [19]:
y_pred

array([2, 2, 2, ..., 2, 0, 1])

In [20]:
print(accuracy_score(y_test, y_pred))


0.7212336892052195


In [8]:
# Clean the training data
df_train['tweet_content'] = df_train['tweet_content'].apply(clean_tweet)

# Clean the test data
df_test['tweet_content'] = df_test['tweet_content'].apply(clean_tweet)

df_train['tweet_content'] = df_train['tweet_content'].str.lower()
df_test['tweet_content'] = df_test['tweet_content'].str.lower()
# Drop rows where 'OriginalTweet' is empty
df_train = df_train.dropna(subset=['tweet_content'])
df_test = df_test.dropna(subset=['tweet_content'])

# Alternatively, if there are rows with just whitespace, use this to remove them as well:
df_train = df_train[df_train['tweet_content'].str.strip() != '']
df_test = df_test[df_test['tweet_content'].str.strip() != '']

df_train = df_train.drop_duplicates(subset=['tweet_content'])
# Display the updated dataframe
df_train.head()

,tweetID,entity,sentiment,tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...


In [9]:
# Separate features and sentiment for training data
train_texts = df_train['tweet_content'].values
train_labels = df_train['sentiment'].values

# Separate features and sentiment for test data
test_texts = df_test['tweet_content'].values
test_labels = df_test['sentiment'].values

In [10]:
# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [11]:
# Initialize the tokenizer
tokenizer = Tokenizer()  # You can adjust the number of words

# Fit the tokenizer on the training texts
tokenizer.fit_on_texts(train_texts)
tokenizer.fit_on_texts(test_texts)

In [12]:
# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [13]:
# Maximum Length in X_train_sequences
maxlen = max(len(tokens) for tokens in train_sequences)
print("Maximum sequence length (maxlen):", maxlen)

Maximum sequence length (maxlen): 166


In [14]:
train_padded = pad_sequences(train_sequences, maxlen=maxlen, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen, padding='post')

In [15]:
# Embedding Input Size
input_size = np.max(train_padded) + 1
input_size

29385

In [16]:
# Define the model
model = Sequential()

# Add an embedding layer 
model.add(Embedding(input_dim=input_size, output_dim=100, input_shape=(56,)))

# Add a bidirectional LSTM layer with 128 units
model.add(Bidirectional(LSTM(128, kernel_regularizer=l2(0.1), return_sequences=True, recurrent_regularizer=l2(0.1))))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Add another LSTM layer
model.add(Bidirectional(LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
# Add batch normalization layer
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Add a dense layer with 64 units and ReLU activation
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))

# Add dropout regularization
model.add(Dropout(0.5))

# Add the output layer with 5 units for 5 labels and softmax activation
model.add(Dense(5, activation='softmax'))

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 56, 100)        │     2,938,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 56, 256)        │       234,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 56, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 56, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,347,465 (12.77 MB)

 Trainable params: 3,346,697 (12.77 MB)

 Non-trainable params: 768 (3.00 KB)

In [18]:
import keras
print(keras.__version__)


3.6.0


In [19]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [21]:
history = model.fit(
    train_padded,
    train_labels_encoded ,
    validation_data=(test_padded,test_labels_encoded),
    epochs=10
)

Epoch 1/10
2075/2075 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step - accuracy: 0.7846 - loss: 0.7121

InvalidArgumentError: Graph execution error:

Detected at node sequential_1/embedding_1/GatherV2 defined at (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 603, in run_forever

  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\asyncio